In [2]:
import pandas as pd 
import numpy as np 
from collections import defaultdict

In [5]:
# Read the dataset
data = pd.read_csv('Pure_Stat.csv')

# Process the data
fighters = defaultdict(lambda: defaultdict(int))

for _, row in data.iterrows():
    for f, w in [('1', 'Fighter 1'), ('2', 'Fighter 2')]:
        fighter = row[w]
        fighters[fighter]['Weight Class'] = row['Weight Class']
        fighters[fighter]['Total Fights'] += 1
        fighters[fighter]['Total Strikes Thrown'] += row[f'F{f} Total Strikes']
        fighters[fighter]['Total Sig Strikes'] += row[f'F{f} Sig Strikes']
        fighters[fighter]['Total Time Elapsed'] += row['Time Elapsed']
        fighters[fighter]['Total Takedown Attempt'] += row[f'F{f} TDA']
        fighters[fighter]['Total Takedown Landed'] += row[f'F{f} TDL']
        fighters[fighter]['Total Knockdowns'] += row[f'F{f} KD']
        fighters[fighter]['Total Control Time'] += row[f'F{f} Control Time']

        if row['Winner'] == f'F{f}':
            fighters[fighter]['Wins'] += 1
            if row['Decision'] == 1:
                fighters[fighter]['Decision Wins'] += 1
            if row['Submission'] == 1:
                fighters[fighter]['Submission Wins'] += 1
            if row['KO/TKO'] == 1:
                fighters[fighter]['KO/TKO Wins'] += 1
        else:
            fighters[fighter]['Losses'] += 1
            if row['Decision'] == 1:
                fighters[fighter]['Decision Losses'] += 1
            if row['Submission'] == 1:
                fighters[fighter]['Submission Losses'] += 1
            if row['KO/TKO'] == 1:
                fighters[fighter]['KO/TKO Losses'] += 1

# Make sure that there are no blank values for wins and losses
for fighter in fighters:
    if 'Wins' not in fighters[fighter]:
        fighters[fighter]['Wins'] = 0
    if 'Losses' not in fighters[fighter]:
        fighters[fighter]['Losses'] = 0
    if 'Decision Wins' not in fighters[fighter]:
        fighters[fighter]['Decision Wins'] = 0

    # Calculate Strikes Thrown Per Minute
    strikes_per_minute = (fighters[fighter]['Total Strikes Thrown'] * 60) / fighters[fighter]['Total Time Elapsed']
    fighters[fighter]['Strikes Thrown Per Minute'] = strikes_per_minute
    # Calculate Sig Strikes Landed Per Minute
    sig_strikes_per_minute = (fighters[fighter]['Total Sig Strikes'] * 60) / fighters[fighter]['Total Time Elapsed']
    fighters[fighter]['Sig Strikes Per Minute'] = sig_strikes_per_minute
    # Calculate TDA Per Minute
    tda_per_minute = (fighters[fighter]['Total Takedown Attempt'] * 60) / fighters[fighter]['Total Time Elapsed']
    fighters[fighter]['TDA/Min'] = tda_per_minute
    # Calculate TDL Per Minute
    tdl_per_minute = (fighters[fighter]['Total Takedown Landed'] * 60) / fighters[fighter]['Total Time Elapsed']
    fighters[fighter]['TDL/Min'] = tdl_per_minute
    # Calculate KD Per Minute
    kd_per_minute = (fighters[fighter]['Total Knockdowns'] * 60) / fighters[fighter]['Total Time Elapsed']
    fighters[fighter]['KD/Min'] = kd_per_minute
    #Calculate Control Time per Minute
    cntrl_prop = (fighters[fighter]['Total Control Time']) / fighters[fighter]['Total Time Elapsed']
    fighters[fighter]['Cntrl Proportion'] = cntrl_prop

# Create a new dataframe and save it as a CSV
fighters_df = pd.DataFrame(fighters).transpose()
fighters_df.to_csv('fighters.csv', index_label='Name')


In [3]:
# Read the dataset
data = pd.read_csv('Pure_Stat.csv')

# Group by weight class
grouped_data = data.groupby('Weight Class').agg({
    'Date': 'count',
    'Time Elapsed': 'sum',
    'F1 Total Strikes': 'sum',
    'F2 Total Strikes': 'sum',
    'F1 Sig Strikes': 'sum',
    'F2 Sig Strikes': 'sum',
    'F1 TDA': 'sum',
    'F2 TDA': 'sum',
    'F1 TDL': 'sum',
    'F2 TDL': 'sum',
    'F1 KD': 'sum',
    'F2 KD': 'sum',
    'Decision': 'sum',
    'KO/TKO': 'sum',
    'Submission': 'sum',
}).reset_index()

# Calculate totals for strikes, TDA, TDL, and KD
grouped_data['Total Strikes'] = grouped_data['F1 Total Strikes'] + grouped_data['F2 Total Strikes']
grouped_data['Total Sig Strikes'] = grouped_data['F1 Sig Strikes'] + grouped_data['F2 Sig Strikes']
grouped_data['Total TDA'] = grouped_data['F1 TDA'] + grouped_data['F2 TDA']
grouped_data['Total TDL'] = grouped_data['F1 TDL'] + grouped_data['F2 TDL']
grouped_data['Total KD'] = grouped_data['F1 KD'] + grouped_data['F2 KD']

# Select desired columns
grouped_data = grouped_data[['Weight Class', 'Date', 'Time Elapsed', 'Total Strikes', 'Total Sig Strikes', 'Total TDA', 'Total TDL', 'Total KD', 'Decision', 'KO/TKO', 'Submission']]

# Rename columns
grouped_data.columns = ['Weight Class', 'Count of Fights', 'Total Time Elapsed', 'Total Strikes Thrown', 'Total Sig Strikes', 'Total TDA', 'Total TDL', 'Total KD', 'Total Decisions', 'Total KOs', 'Total Submissions']

# Save the grouped data as a CSV
grouped_data.to_csv('averages.csv', index=False)


In [4]:
import pandas as pd

# Read data
fighters = pd.read_csv("fighters.csv")
pure_stat = pd.read_csv("Pure_Stat.csv")

# Initialize new columns with default values
fighters["Sig Strikes Absorbed per Minute"] = 0.0
fighters["Takedowns Faced per Minute"] = 0.0
fighters["Takedowns Suffered per Minute"] = 0.0

# Calculate new column values
for index, fighter in fighters.iterrows():
    fighter_name = fighter["Name"]

    # Filter fights where fighter is F1 or F2
    f1_fights = pure_stat[pure_stat["Fighter 1"] == fighter_name]
    f2_fights = pure_stat[pure_stat["Fighter 2"] == fighter_name]

    # Calculate Sig Strikes Absorbed per Minute
    absorbed_f1 = f1_fights["F2 Sig Strikes"].sum()
    absorbed_f2 = f2_fights["F1 Sig Strikes"].sum()
    total_absorbed = absorbed_f1 + absorbed_f2

    # Calculate Takedowns Faced per Minute
    takedowns_faced_f1 = f1_fights["F2 TDA"].sum()
    takedowns_faced_f2 = f2_fights["F1 TDA"].sum()
    total_takedowns_faced = takedowns_faced_f1 + takedowns_faced_f2

    # Calculate Takedowns Suffered per Minute
    takedowns_suffered_f1 = f1_fights["F2 TDL"].sum()
    takedowns_suffered_f2 = f2_fights["F1 TDL"].sum()
    total_takedowns_suffered = takedowns_suffered_f1 + takedowns_suffered_f2

    # Calculate total time elapsed
    total_time_elapsed = f1_fights["Time Elapsed"].sum() + f2_fights["Time Elapsed"].sum()

    # Calculate per minute values
    fighters.loc[index, "Sig Strikes Absorbed per Minute"] = (total_absorbed / total_time_elapsed) * 60
    fighters.loc[index, "Takedowns Faced per Minute"] = (total_takedowns_faced / total_time_elapsed) * 60
    fighters.loc[index, "Takedowns Suffered per Minute"] = (total_takedowns_suffered / total_time_elapsed) * 60

# Save the updated DataFrame to a new CSV file
fighters.to_csv("fighters.csv", index=False)


In [5]:
# Load the dataset
df = pd.read_csv('fighters.csv')

# Replace any blank values with 0
df_filled = df.fillna(0)

# Save the filled DataFrame to a new CSV file
df_filled.to_csv('fighters.csv', index=False)

In [6]:
fighters = pd.read_csv('fighters.csv')

# Specify the columns to remove
cols_to_remove = ['Total Time Elapsed', 'Total Takedown Attempt', 'Total Strikes Thrown', 'Total Sig Strikes', 'Total Takedown Landed', 'Total Knockdowns']

# Drop the specified columns
fighters = fighters.drop(columns=cols_to_remove)
fighters.to_csv('fighters.csv', index=False)


In [7]:
# Load the DataFrame with all columns
fighters1 = pd.read_csv('fighters.csv')

# Define the desired column order
new_order = ['Name', 'Weight Class', 'Total Fights', 'Wins', 'Losses', 'Strikes Thrown Per Minute', 'Sig Strikes Per Minute', 'Sig Strikes Absorbed per Minute', 'TDA/Min','TDL/Min','KD/Min','Takedowns Faced per Minute','Takedowns Suffered per Minute','Decision Wins', 'KO/TKO Wins', 'Submission Wins', 'Decision Losses', 'KO/TKO Losses', 'Submission Losses']

# Reorder the DataFrame columns
fighters1 = fighters1[new_order]
fighters1.to_csv('fighters.csv', index=False)

In [8]:
# Read the dataset
data = pd.read_csv('fighters.csv')

# Convert columns to proper data types
numeric_columns = data.columns[2:]
data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Group by Weight Class and calculate the mean
mean_df = data.groupby('Weight Class').mean()

# Calculate the standard deviation for each group
std_df = data.groupby('Weight Class').std()

# Calculate average + 1.5 std deviations
plus_15std_df = mean_df + 1.5 * std_df

# Calculate average - 1.5 std deviations
minus_15std_df = mean_df - 1.5 * std_df

# Save mean_df to a CSV file
mean_df.to_csv('mean_per_weight_class.csv')

# Save plus_15std_df to a CSV file
plus_15std_df.to_csv('mean_plus_1.5std_per_weight_class.csv')

# Save minus_15std_df to a CSV file
minus_15std_df.to_csv('mean_minus_1.5std_per_weight_class.csv')
